## 2.기출문제 (빅분기 실기 4회)

In [ ]:
# 데이터 다운로드

In [ ]:
!git clone https://github.com/Soyoung-Yoon/bigdata

#### 2-1. 신상 데이터
age 컬럼에 대해 아래의 과정을 수행하고 (다) 단계의 결과값을 출력하여라.
- (가) 제 1사분위수와 제 3사분위수를 구하기
- (나) 두 값의 차이 절댓값 구하기
- (다) 그 값의 소수점 버리기


In [2]:
import pandas as pd
df = pd.read_csv('./bigdata/basic1_data04.csv')

In [3]:
Q1 = df['age'].quantile(0.25)
Q3 = df['age'].quantile(0.75)
나 = Q3 - Q1
int(나)

50

#### 2-2. Facebook 데이터

Facebook 데이터셋을 불러와 (loves반응 + wows반응)/(reactions반응) 비율이 0.4보다 크고 0.5보다 작으면서, type이 'video'인 데이터의 갯수를 구하여라
- 'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/fb_data04.csv' 파일 사용

In [4]:
df = pd.read_csv('./bigdata/fb_data04.csv')
df.head()

,id,type,reactions,comments,shares,likes,loves,wows,hahas,sads,angrys
0,1,video,529,512,262,432,92,3,1,1,0
1,2,photo,150,0,0,150,0,0,0,0,0
2,3,video,227,236,57,204,21,1,1,0,0
3,4,photo,111,0,0,111,0,0,0,0,0
4,5,photo,213,0,0,204,9,0,0,0,0


In [5]:
df['ratio'] = (df['loves'] + df['wows']) / df['reactions']
len(df[ ( df['ratio'] > 0.4 ) & ( df['ratio'] < 0.5 ) & ( df['type']=="video" ) ])

90

#### 2-3. 넷플릭스 데이터 
netflix_subset 데이터셋에서 date_added가 2018년 1월이면서 country가 United Kingdom 단독 제작인 데이터의 개수를 구하여라
- 'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/nf_data04.csv' 파일 사용

In [6]:
df = pd.read_csv('./bigdata/nf_data04.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."


In [15]:
df['date_added'] = pd.to_datetime(df['date_added'])
df['month'] = df['date_added'].dt.month
df['year'] = df['date_added'].dt.year
len(df[( df['year']==2018 )&(df['month']==1)&(df['country']=="United Kingdom")])

6

In [63]:

cond1 = df['date_added'].str.contains('January')
cond2 = df['date_added'].str.contains('2018')
cond3 = df['country'] == 'United Kingdom'

len(df[cond1 & cond2 & cond3])

6

## 3.기출문제 (빅분기 실기 4회)
자동차 시장 세분화
- 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
- 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!
- 예측할 값(y): "Segmentation" (1,2,3,4)
- 평가: Macro f1-score
- 제출형식

        ID,Segmentation
        458989,1
        458994,2
        459000,3
        459003,4

- 다음의 데이터 파일 준비를 실행해서 x_train.csv, y_train.csv, x_test.csv 파일을 준비합니다.


### 3-01.데이터 불러오기, 탐색

In [16]:
# 데이터 파일 준비
# 그냥 실행해서 사용 파일을 생성하도록 합니다.
import pandas as pd

train = pd.read_csv('./bigdata//train_04.csv')
test = pd.read_csv('./bigdata//test_04.csv')
train.head(2)
X_train = train.iloc[:, :-1]
Y_train = train[['ID', 'Segmentation']]
X_train.to_csv('X_train.csv', index=False)
Y_train.to_csv('Y_train.csv', index=False)
test.to_csv('X_test.csv', index=False)
print('파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !')

파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !


In [35]:
# [0] 사용 라이브러리 import
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [43]:
# [1] x_train.csv, y_train.csv, x_test.csv 읽어오기
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('Y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [44]:
print(X_train.info())
print("="*90)
print(y_train.info())
print("="*90)
print(X_test.info())
print("="*90)

print(X_train.describe())
print("="*90)
print(y_train.describe())
print("="*90)
print(X_test.describe())
print("="*90)

print(X_train.isna().sum().sum())
print("="*90)
print(y_train.isna().sum().sum())
print("="*90)
print(X_test.isna().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 520.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   ID            6665 non-null   int64
 1   Segmentation  6665 non-null   int64
dt

In [45]:
X_train = X_train.drop(columns = "ID") 
X_test_ID = X_test.pop('ID') 
y_train = y_train.drop(columns = "ID") 

display(X_train)
display(y_train)
display(X_test)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
2,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
3,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6
4,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6
...,...,...,...,...,...,...,...,...,...
6660,Male,Yes,41,Yes,Artist,0.0,High,5.0,Cat_6
6661,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4
6662,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6
6663,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6


,Segmentation
0,4
1,2
2,2
3,3
4,3
...,...
6660,2
6661,4
6662,4
6663,2


,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
3,Male,Yes,47,Yes,Doctor,0.0,High,5.0,Cat_4
4,Male,Yes,61,Yes,Doctor,5.0,Low,3.0,Cat_6
...,...,...,...,...,...,...,...,...,...
2149,Female,No,35,Yes,Entertainment,1.0,Low,2.0,Cat_6
2150,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6
2151,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6
2152,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4


In [46]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
le = LabelEncoder()

ls = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']

X_train['Gender'] = le.fit_transform(X_train['Gender'])
X_train['Ever_Married'] = le.fit_transform(X_train['Ever_Married'])
X_train['Graduated'] = le.fit_transform(X_train['Graduated'])
X_train['Profession'] = le.fit_transform(X_train['Profession'])
X_train['Spending_Score'] = le.fit_transform(X_train['Spending_Score'])
X_train['Var_1'] = le.fit_transform(X_train['Var_1'])

X_test['Gender'] = le.fit_transform(X_test['Gender'])
X_test['Ever_Married'] = le.fit_transform(X_test['Ever_Married'])
X_test['Graduated'] = le.fit_transform(X_test['Graduated'])
X_test['Profession'] = le.fit_transform(X_test['Profession'])
X_test['Spending_Score'] = le.fit_transform(X_test['Spending_Score'])
X_test['Var_1'] = le.fit_transform(X_test['Var_1'])


# scaler = MinMaxScaler()

# ls = ['Age']

# X_train[ls] = scaler.fit_transform(X_train[ls])
# X_test[ls] = scaler.fit_transform(X_test[ls])

In [47]:
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X_train, y_train, test_size=0.3, stratify = y_train)

In [52]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(1234, max_depth = 8)
modelRF.fit(X_tr, y_tr)
modelRF.score(X_ts, y_ts)

y_pred = modelRF.predict(X_test)

C:\Users\inseo\AppData\Local\Temp\ipykernel_17024\2219406668.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelRF.fit(X_tr, y_tr)


In [53]:
pd.DataFrame({"ID":X_test_ID, "Segmentation":y_pred})

,ID,Segmentation
0,458989,1
1,458994,2
2,459000,3
3,459003,2
4,459005,1
...,...,...
2149,467950,1
2150,467954,4
2151,467958,1
2152,467961,2
